In [11]:
import numpy as np

# Fix these ranges, not all is interesting everywhere
mus = np.arange(0, 0.7, 0.1)  
deltas = np.array([0.01, 0.001, 0.0001, 0.00001])
epsilons = np.arange(0.7, 1.1, 0.1)  
alphas = np.arange(0.1, 1.1, 0.1)


print(f'Mus: {mus}')
print(f'Deltas: {deltas}')
print(f'Epsilons: {epsilons}')
print(f'Alphas: {alphas}')

# for mu in mus:
#     for delta in deltas:
#         for epsilon in epsilons:
#             for alpha in alphas:

#                 # Check for time T where the system's clusteredness is stationary
#                 print('hi')

print(f'num iterations: {len(mus)*len(deltas)*len(epsilons)*len(alphas)}')


Mus: [0.  0.1 0.2 0.3 0.4 0.5 0.6]
Deltas: [1.e-02 1.e-03 1.e-04 1.e-05]
Epsilons: [0.7 0.8 0.9 1.  1.1]
Alphas: [0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1. ]
num iterations: 1400


In [2]:
# I CAN'T HAVE MU 0 DUMBASSSSSSS

from ParticleSystem import ParticleSystem
import pandas as pd
import numpy as np 

WORLD_WIDTH = 300 # 90 # 200 # 20
WORLD_HEIGHT = 300 # 90 # 200 # 20
DELTA = 0.0001 # 0.001 # 0.01 # 0.1 # probability to change direction
MUS = np.arange(0.1, 0.7, 0.1)  
EPSILON = 0.9 # probability to just follow the nomral direction for one iteration (and not randomly move)
ALPHA = 0.8 # probability to be an active (normal) particle
DOT_SIZE = 3 # 10 # 4
MIDDLE_CLUSTER_SIZE = -1 # 10
NUM_ITERATIONS =  100000000000000000000 # None # 10000
INIT_REFRESH_RATE = 10 # 100
INIT_PAUSED_STATUS = False

results = []

for MU in MUS:

    world = ParticleSystem(
        WORLD_WIDTH,
        WORLD_HEIGHT,
        DELTA,
        MU,
        EPSILON, 
        ALPHA,
        DOT_SIZE,
        MIDDLE_CLUSTER_SIZE,
        NUM_ITERATIONS,
        INIT_REFRESH_RATE,
        INIT_PAUSED_STATUS
    )

    T = world.run_simulation()

    results.append({'mu': MU, 'T': T})
    print(f'Done for mu = {MU}, T = {T}')

results_df = pd.DataFrame(results)

results_df.to_csv('latest_latest_run.csv')

KeyboardInterrupt: 

: 

In [6]:
# so maybe in a first instance we just vary mu, see how the plots come then vary other parameters
# the plot of the data will be mu on the x axis and T on the y axis where T is the time where the system becomes 'converged'
import numpy as np
from ParticleSystem import ParticleSystem
import pandas as pd 

mus = np.arange(0, 0.7, 0.1)  
delta = 0.0001
epsilon = 0.9
alpha = 0.8

results = []

for mu in mus:
    world = ParticleSystem(
        300, # width
        300, # height
        delta,
        mu,
        epsilon, 
        alpha,
        3, # dot_size
        -1, # middle cluster size
        1000000000000000000000000000, # num iterations
        150, # init refresh rate
        False # init paused status
    )

    T = world.run_simulation()

    results.append({'mu': mu, 'T': T})
    print(f'Done for mu = {mu}, T = {T}')

results_df = pd.DataFrame(results)

results_df.to_csv('latest_latest_run.csv')
    
              

IndexError: Cannot choose from an empty sequence

In [1]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('latest_run_cluster_records.csv')
# print(df.head())

fig = px.line(df, x='Iteration', y='Cluster Cardinality', title='Cluster Cardinality over Time')
# fig.update_layout()
fig.show()

In [12]:
import numpy as np
import plotly.graph_objects as go

# Toroidal parameters
R = 2      # Major radius (distance from center to middle of tube)
r = 0.6    # Minor radius (radius of tube cross-section)
N_u = 10   # Number of lattice points around the major loop
N_v = 10   # Number of lattice points around the minor loop

# Generate discrete lattice points
theta = np.linspace(0, 2*np.pi, N_u, endpoint=False)  # Major angle (around torus)
phi = np.linspace(0, 2*np.pi, N_v, endpoint=False)    # Minor angle (tube cross-section)
theta, phi = np.meshgrid(theta, phi)

# Convert to Cartesian coordinates
X = (R + r * np.cos(phi)) * np.cos(theta)
Y = (R + r * np.cos(phi)) * np.sin(theta)
Z = r * np.sin(phi)

# Convert to 1D arrays for easier edge connections
X = X.flatten()
Y = Y.flatten()
Z = Z.flatten()

# Create Plotly figure
fig = go.Figure()

# Add lattice points
fig.add_trace(go.Scatter3d(x=X, y=Y, z=Z, mode='markers', marker=dict(size=4, color='blue'), name='Lattice Points'))

# Add connections between neighboring points
for i in range(N_u):
    for j in range(N_v):
        idx = i * N_v + j  # Index in the flattened array
        
        # Connect to neighbor along major direction (theta)
        next_i = ((i + 1) % N_u) * N_v + j
        fig.add_trace(go.Scatter3d(
            x=[X[idx], X[next_i]], y=[Y[idx], Y[next_i]], z=[Z[idx], Z[next_i]], 
            mode='lines', line=dict(color='black', width=2), showlegend=False))
        
        # Connect to neighbor along minor direction (phi)
        next_j = i * N_v + (j + 1) % N_v
        fig.add_trace(go.Scatter3d(
            x=[X[idx], X[next_j]], y=[Y[idx], Y[next_j]], z=[Z[idx], Z[next_j]], 
            mode='lines', line=dict(color='black', width=2), showlegend=False))

# Add transparent planes to make the torus stand out
plane_size = 5  # Adjust plane size for better visibility
plane_opacity = 0.2  # Adjust transparency

# XY Plane
fig.add_trace(go.Surface(
    z=np.full((2, 2), 0),  # Flat plane at z=0
    x=[-plane_size, plane_size],
    y=[-plane_size, plane_size],
    colorscale=[(0, 'lightgrey'), (1, 'lightgrey')],
    opacity=plane_opacity,
    showscale=False
))

# XZ Plane
fig.add_trace(go.Surface(
    y=np.full((2, 2), 0),  # Flat plane at y=0
    x=[-plane_size, plane_size],
    z=[-plane_size, plane_size],
    colorscale=[(0, 'lightgrey'), (1, 'lightgrey')],
    opacity=plane_opacity,
    showscale=False
))

# YZ Plane
fig.add_trace(go.Surface(
    x=np.full((2, 2), 0),  # Flat plane at x=0
    y=[-plane_size, plane_size],
    z=[-plane_size, plane_size],
    colorscale=[(0, 'lightgrey'), (1, 'lightgrey')],
    opacity=plane_opacity,
    showscale=False
))

# Update layout for better viewing
fig.update_layout(
    title="Interactive Discrete Toroidal Lattice with Transparent Planes",
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the interactive plot
fig.show()

